In [2]:
import torchvision.models as models
import torchinfo
import torch
# 加載 GoogLeNet 模型
model = models.alexnet(pretrained=True)
print(model)

input_shape = (3, 224, 224)

/home/popo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/popo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
print('==================================================================================================================================================')
print('= HW 2-1-1 Calculate the number of model parameters                                                                                              =')
print('==================================================================================================================================================')
# Calculating the total number of parameters in the model
total_params = sum(param.numel() for param in model.parameters())
print("Total number of parameters: ", total_params)

= HW 2-1-1 Calculate the number of model parameters                                                                                              =
Total number of parameters:  61100840


In [4]:
print('==================================================================================================================================================')
print('= HW 2-1-2 Calculate memory requirements for storing the model weights                                                                           =')
print('==================================================================================================================================================')
param_size = sum(param.numel() * param.element_size() for param in model.parameters())
print("Total memory requirement: {} MB".format(param_size/(1024*1024)))

= HW 2-1-2 Calculate memory requirements for storing the model weights                                                                           =
Total memory requirement: 233.08120727539062 MB


In [5]:
print('==================================================================================================================================================')
print('= HW 2-1-3 Use Torchinfo to print model architecture including the number of parameters and the output activation size of each layer             =')
print('==================================================================================================================================================')
# torchinfo.summary(model, (3, 224, 224), batch_dim=0, col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"), verbose=0)
torchinfo.summary(model, (3, 224, 224), batch_dim=0, col_names=("output_size", "num_params"), verbose=0)

= HW 2-1-3 Use Torchinfo to print model architecture including the number of parameters and the output activation size of each layer             =


In [ ]:
def calculate_output_shape(input_shape, layer):
    # Calculate the output shape for Conv2d, MaxPool2d, and Linear layers
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
        kernel_size = (
            layer.kernel_size
            if isinstance(layer.kernel_size, tuple)
            else (layer.kernel_size, layer.kernel_size)
        )
        stride = (
            layer.stride
            if isinstance(layer.stride, tuple)
            else (layer.stride, layer.stride)
        )
        padding = (
            layer.padding
            if isinstance(layer.padding, tuple)
            else (layer.padding, layer.padding)
        )
        dilation = (
            layer.dilation
            if isinstance(layer.dilation, tuple)
            else (layer.dilation, layer.dilation)
        )

        output_height = (
            input_shape[1] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1
        ) // stride[0] + 1
        output_width = (
            input_shape[2] + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1
        ) // stride[1] + 1
        return (
            layer.out_channels if hasattr(layer, "out_channels") else input_shape[0],
            output_height,
            output_width,
        )
    elif isinstance(layer, nn.Linear):
        # For Linear layers, the output shape is simply the layer's output features
        return (layer.out_features,)
    else:
        return input_shape


def calculate_macs(layer, input_shape, output_shape):
    # Calculate MACs for Conv2d and Linear layers
    if isinstance(layer, nn.Conv2d):
        kernel_ops = (
            layer.kernel_size[0]
            * layer.kernel_size[1]
            * (layer.in_channels / layer.groups)
        )
        output_elements = output_shape[1] * output_shape[2]
        macs = int(kernel_ops * output_elements * layer.out_channels)
        return macs
    elif isinstance(layer, nn.Linear):
        # For Linear layers, MACs are the product of input features and output features
        macs = int(layer.in_features * layer.out_features)
        return macs
    else:
        return 0

In [ ]:
import torch.nn as nn
input_shape = (3, 224, 224)
total_mac = 0

# Initial input shape
input_shape = (3, 224, 224)
total_macs = 0

# Iterate through the layers of the model
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d, nn.ReLU, nn.Linear)):
        output_shape = calculate_output_shape(input_shape, layer)
        macs = calculate_macs(layer, input_shape, output_shape)
        total_macs += macs
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: {macs}"
            )
        elif isinstance(layer, nn.MaxPool2d):
            # Also print shape transformation for MaxPool2d layers (no MACs calculated)
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: N/A"
            )
        input_shape = output_shape  # Update the input shape for the next layer
print('==================================================================================================================================================')
print('= HW 2-1-4 Calculate computation requirements                                                                                                    =')
print('==================================================================================================================================================')
print(f"Total MACs: {total_macs}")

Layer: conv1.conv, Type: Conv2d, Input Shape: (3, 224, 224), Output Shape: (64, 112, 112), MACs: 118013952
Layer: maxpool1, Type: MaxPool2d, Input Shape: (64, 112, 112), Output Shape: (64, 55, 55), MACs: N/A
Layer: conv2.conv, Type: Conv2d, Input Shape: (64, 55, 55), Output Shape: (64, 55, 55), MACs: 12390400
Layer: conv3.conv, Type: Conv2d, Input Shape: (64, 55, 55), Output Shape: (192, 55, 55), MACs: 334540800
Layer: maxpool2, Type: MaxPool2d, Input Shape: (192, 55, 55), Output Shape: (192, 27, 27), MACs: N/A
Layer: inception3a.branch1.conv, Type: Conv2d, Input Shape: (192, 27, 27), Output Shape: (64, 27, 27), MACs: 8957952
Layer: inception3a.branch2.0.conv, Type: Conv2d, Input Shape: (64, 27, 27), Output Shape: (96, 27, 27), MACs: 13436928
Layer: inception3a.branch2.1.conv, Type: Conv2d, Input Shape: (96, 27, 27), Output Shape: (128, 27, 27), MACs: 80621568
Layer: inception3a.branch3.0.conv, Type: Conv2d, Input Shape: (128, 27, 27), Output Shape: (16, 27, 27), MACs: 2239488
Layer: i

In [ ]:
import torch
import torchvision.models as models

activation = {}
# Define a hook function
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# 加載 GoogLeNet 模型
model = models.googlenet(pretrained=True)
input_shape = (64, 3, 224, 224)
input_tensor = torch.randn(input_shape)

In [ ]:
model.eval()
output = model(input_tensor)
for layer_name, layer in model.named_modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.register_forward_hook(get_activation(layer_name))
# Access the saved activations
for layer in activation:
    print(layer)
    print(f"Activation from layer {layer}: {activation[layer].shape}")